<a href="https://colab.research.google.com/github/banghj-kr/python/blob/gScore/gScore_01_Data_Setting_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 용어 설명

1. **골프 코스 (Golf Course)**: 골프를 할 수 있는 코스나 경기장.
2. **골프 클럽 (Golf Club, GC)**: 골프장을 운영하는 클럽이나 클럽하우스.
3. **전반 코스 (Front Nine, F9)**: 18홀 중 앞의 9홀.
4. **후반 코스 (Back Nine, B9)**: 18홀 중 뒤의 9홀.
5. **전체 스코어 (Total Score)**: 전체 라운드의 총 타수.
6. **전체 스코어에서 정규 타수를 뺀 점수 (Score Against Par, S.A.P.)**: 전체 스코어에서 파를 뺀 점수.
7. **정규 타수 (Par)**: 각 홀의 기준 타수.
8. **퍼팅수가 아닌 타수 (Strokes or Shots)**: 퍼팅이 아닌 나머지 모든 타수.
9. **페널티 타수 (Penalty Strokes)**: 페널티로 추가된 타수.
10. **퍼팅 수 (Putts)**: 그린에서의 퍼팅 타수.
11. **페어웨이 적중률 (Fairways Hit, FW Hit)**: 티 샷이 페어웨이에 적중한 비율.
12. **그린 적중률 (Greens in Regulation, GiR)**: 기준 타수 내에 그린에 도달한 비율.
13. **스크램블링 (Scrambling)**: 그린에 도달하지 못한 상황에서 파 또는 그 이하의 스코어를 기록한 비율.
14. **스코어 용어 (Scoring Terms)**

# 01. 라이브러리 로딩 및 함수 정의

In [ ]:
# 01. 필요한 라이브러리 로딩 및 함수 정의 (240808)
print("01. 필요한 라이브정의 로딩 및 함수 정의")
#!pip install gspread google-auth
# 필요한 라이브러리 로딩
import pandas as pd
import numpy as np
import os
from datetime import datetime
import traceback
import pytz
from IPython import get_ipython

def check_environment():
    try:
        if 'google.colab' in env:  # Colab 환경
            from google.colab import data_table
            import gspread
            from google.auth.transport.requests import Request
            from google.oauth2.service_account import Credentials
            from google.colab import drive

            # 구글 드라이브 마운트 - Colab 환경 only
            drive.mount('/content/drive')

            # Google Sheets API 인증 설정 - Colab 환경 only
            scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
            creds = Credentials.from_service_account_file('/content/drive/MyDrive/Colab Notebooks/banghj-python-api-2449c408ca20.json', scopes=scope)
            client = gspread.authorize(creds)

            # 데이터프레임을 구글 코랩의 데이터 테이블로 출력 활성화 - Colab 환경 only
            data_table.enable_dataframe_formatter()

            # 기본 경로 설정
            base_path = '/content/drive/MyDrive/Colab Notebooks/gScore/'
            return base_path, client

        elif 'ZMQInteractiveShell' in env:  # Jupyter 환경
            # 기본 경로 설정
            base_path = ''
            return base_path, None
        else:
            return None, None
    except ImportError:
        return None, None

# 서울 시간대 설정
tz = pytz.timezone('Asia/Seoul')

# 환경에 맞는 기본 경로 설정
env = str(get_ipython())  # 실행 환경 확인
base_path, client = check_environment()
print(f"  - 기본 경로를 '{base_path}'로 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# pandas display 옵션 설정
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 100)

# 필요한 함수 정의

# Scoring Terms를 계산하는 함수 정의
def calculate_st(score, sap, par):
    scoring_terms = {
        1: 'Hole-in-One',
        -3: 'Albatross',
        -2: 'Eagle',
        -1: 'Birdie',
        0: 'Par',
        1: 'Bogey',
        2: 'Double Bogey',
        3: 'Triple Bogey',
        4: 'Quadruple Bogey'
    }
    if score >= par * 2:
        return 'Double Par'
    return scoring_terms.get(sap, 'Unknown')

# 데이터프레임을 CSV 파일로 저장하는 함수 정의
def save_to_csv(df, output_file_path):
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    df.to_csv(output_file_path, encoding='utf-8-sig', index=False)
    print(f"  * '{output_file_path}' 파일에 데이터를 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 파일 경로 및 이름 설정 함수 정의
def generate_file_path(mode, base_path, sub_dir, file_name, ext, add_timestamp=True):
    if mode == 'input':
        # 입력 파일 경로 설정 (타임스탬프 없이)
        return f'{base_path}{sub_dir}/{file_name}.{ext}'
    elif mode == 'output':
        # 출력 파일 경로 설정 (타임스탬프 추가 가능)
        output_file_path = f'{base_path}{sub_dir}/{file_name}.{ext}'
        if add_timestamp:
            date_str = datetime.now(tz).strftime("%Y%m%d.%H%M%S")
            output_file_path = f"{os.path.splitext(output_file_path)[0]}_{date_str}.{ext}"
        return output_file_path
    else:
        raise ValueError("  * 파일의 경로와 이름을 설정하는 함수의 첫 번째 매개변수는 'input' 또는 'output'으로 설정해주세요.")

print(f"  - 라이브러리 로딩 및 함수 정의를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

01. 필요한 라이브정의 로딩 및 함수 정의
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  - 기본 경로를 '/content/drive/MyDrive/Colab Notebooks/gScore/'로 설정하였습니다. (2024-08-08 13:06:51)
  - 라이브러리 로딩 및 함수 정의를 완료하였습니다. (2024-08-08 13:06:51)


# 02. Data 입력 및 설정

## 02-1. Course Data

In [ ]:
# 02-1. Course Data 입력 및 설정 (240621)
print("02-1. Course Data 입력 및 설정")

# 파일 경로 및 이름, 시트 이름 설정
input_file_path = generate_file_path('input', base_path, 'Data', 'gScore_InputData', 'xlsx')  # 입력 파일 경로 및 이름 설정
sheet_name = 'Course_Data'  # 시트 이름 설정
output_file_path = generate_file_path('output', base_path, 'Output', '01_Course_Data', 'csv')  # 출력 파일 경로 및 이름 설정

if not os.path.exists(input_file_path):  # 파일 존재 여부 확인
    print(f"  - Error: '{input_file_path}' 파일을 찾을 수 없습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
else:
    try:
        with pd.ExcelFile(input_file_path) as xl:
            print(f"  - '{input_file_path}' 파일을 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
            if sheet_name not in xl.sheet_names:  # 시트 존재 여부 확인
                print(f"  - Error: '{sheet_name}' 시트를 찾을 수 없습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
            else:
                print(f"  - '{sheet_name}' 시트를 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

                # 엑셀 파일을 읽어 데이터프레임으로 변환
                course_df = pd.read_excel(xl, sheet_name=sheet_name, usecols="B:M")

                course_df.dropna(subset=['par_1'], inplace=True)  # 특정 열이 비어있는 행 제거
                print(f"  - Course Data를 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

                # Par1부터 Par9까지의 열을 정수형으로 변환 및 배열로 변환하여 Par 열 추가
                par_columns = [f'par_{i}' for i in range(1, 10)]
                course_df[par_columns] = course_df[par_columns].astype(np.int32)
                course_df['par'] = course_df[par_columns].values.tolist()

                # ParTotal 열 추가
                course_df['par_total'] = course_df['par'].apply(sum)

                # Par1부터 Par9까지의 개별 열 삭제
                course_df.drop(columns=par_columns, inplace=True)

                # 데이터프레임을 GolfClub과 CosName 열 기준으로 오름차순 정렬
                course_df.sort_values(by=['golf_course'], ascending=True, inplace=True)

                # 인덱스 초기화
                course_df.reset_index(drop=True, inplace=True)

                # 수정된 데이터프레임을 CSV 파일로 저장 (UTF-8-BOM으로 인코딩)
                save_to_csv(course_df, output_file_path)

                print(f"  - 데이터를 데이터프레임으로 모두 정리하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    except Exception as e:
        print("Error:", e)
        traceback.print_exc()  # 오류 발생시 오류 내용 출력

    finally:
        xl.close()  # 엑셀 파일 닫기

# 결과 출력
print("\n  * Course DataFrame (course_df):")
display(course_df)

print("\n  * Course DataFrame Information (course_df):")
display(course_df.info())

# ==================================================

02-1. Course Data 입력 및 설정
  - '/content/drive/MyDrive/Colab Notebooks/gScore/Data/gScore_InputData.xlsx' 파일을 읽었습니다. (2024-08-08 13:06:51)
  - 'Course_Data' 시트를 읽었습니다. (2024-08-08 13:06:51)
  - Course Data를 읽었습니다. (2024-08-08 13:06:52)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Output/01_Course_Data_20240808.130651.csv' 파일에 데이터를 저장하였습니다. (2024-08-08 13:06:52)
  - 데이터를 데이터프레임으로 모두 정리하였습니다. (2024-08-08 13:06:52)

  * Course DataFrame (course_df):


,golf_course,course,address,par,par_total
0,360도,IN,경기도 여주시 강천면 부평로 609,"[4, 3, 5, 4, 4, 3, 5, 4, 4]",36
1,360도,OUT,경기도 여주시 강천면 부평로 609,"[4, 4, 5, 3, 4, 4, 3, 5, 4]",36
2,강남300,OUT,경기도 광주시 새말길 353,"[5, 4, 3, 4, 3, 4, 4, 4, 5]",36
3,강남300,IN,경기도 광주시 새말길 353,"[5, 4, 3, 4, 4, 4, 3, 4, 5]",36
4,골프존카운티 사천,다솔,경상남도 사천시 서포면 구송로 151,"[4, 4, 5, 3, 4, 5, 3, 4, 4]",36
...,...,...,...,...,...
138,해비치 제주,SKY,제주특별자치도 서귀포시 남원읍 원님로399번길 319,"[4, 3, 4, 5, 4, 4, 3, 5, 4]",36
139,해비치 제주,PALM,제주특별자치도 서귀포시 남원읍 원님로399번길 319,"[4, 4, 3, 4, 5, 4, 4, 3, 5]",36
140,히든밸리,스카이,충청북도 진천군 백곡면 소토골길 61,"[4, 4, 4, 3, 5, 3, 4, 5, 4]",36
141,히든밸리,밸리,충청북도 진천군 백곡면 소토골길 61,"[5, 4, 4, 3, 4, 3, 5, 4, 4]",36



  * Course DataFrame Information (course_df):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   golf_course  143 non-null    object
 1   course       143 non-null    object
 2   address      143 non-null    object
 3   par          143 non-null    object
 4   par_total    143 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 5.7+ KB


None

## 02-2. Round Data

In [ ]:
# 02-2. Round Data 입력 및 설정 (240621)
print("02-2. Round Data 입력 및 설정")

# 파일 경로 및 이름, 시트 이름 설정
sheet_name = 'Round_Data'  # 시트 이름 설정
output_file_path = generate_file_path('output', base_path, 'Output', '02_Round_Data', 'csv')  # 출력 파일 경로 및 이름 설정

if not os.path.exists(input_file_path):  # 파일 존재 여부 확인
    print(f"  - Error: '{input_file_path}' 파일을 찾을 수 없습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
else:
    try:
        with pd.ExcelFile(input_file_path) as xl:
            print(f"  - '{input_file_path}' 파일을 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
            if sheet_name not in xl.sheet_names:  # 시트 존재 여부 확인
                print(f"  - Error: '{sheet_name}' 시트를 찾을 수 없습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
            else:
                print(f"  - '{sheet_name}' 시트를 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

                round_df = pd.read_excel(xl, sheet_name=sheet_name, usecols="G:CG")  # 엑셀 파일을 읽어 데이터프레임으로 변환
                round_df.dropna(subset=['stroke_f9_1'], inplace=True)  # 특정 열이 비어있는 행 제거
                print(f"  - Round Data를 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

                # companions 배열 생성
                round_df['companions'] = round_df.apply(lambda row: [row['companion_1'], row['companion_2'], row['companion_3']], axis=1)

                sections = ['f9', 'b9']
                for section in sections:
                    round_df[f'stroke_{section}'] = round_df.apply(lambda row: [int(x) if pd.notna(x) else -1 for x in [row[f'stroke_{section}_{i}'] for i in range(1, 10)]], axis=1)
                    round_df[f'stroke_{section}_total'] = round_df[f'stroke_{section}'].apply(sum)
                    round_df[f'putt_{section}'] = round_df.apply(lambda row: [int(x) if pd.notna(x) else -1 for x in [row[f'putt_{section}_{i}'] for i in range(1, 10)]], axis=1)
                    round_df[f'putt_{section}_total'] = round_df[f'putt_{section}'].apply(sum)
                    round_df[f'penalty_{section}'] = round_df.apply(lambda row: [int(x) if pd.notna(x) else -1 for x in [row[f'penalty_{section}_{i}'] for i in range(1, 10)]], axis=1)
                    round_df[f'penalty_{section}_total'] = round_df[f'penalty_{section}'].apply(sum)
                    round_df[f'fw_hit_{section}'] = round_df.apply(lambda row: [1 if row[f'fw_hit_{section}_{i}'] == 'Yes' else (0 if row[f'fw_hit_{section}_{i}'] == 'No' else np.nan) for i in range(1, 10)], axis=1)

                round_df['stroke_total'] = round_df['stroke_f9_total'] + round_df['stroke_b9_total']
                round_df['putt_total'] = round_df['putt_f9_total'] + round_df['putt_b9_total']
                round_df['penalty_total'] = round_df['penalty_f9_total'] + round_df['penalty_b9_total']

                # 개별 열 삭제
                companion_columns = [f'companion_{i}' for i in range(1, 4)]
                round_df.drop(columns=companion_columns, inplace=True)
                for section in sections:
                    round_df.drop(columns=[f'{type}_{section}_{i}' for type in ['stroke', 'putt', 'penalty', 'fw_hit'] for i in range(1, 10)], inplace=True)

                # 수정된 데이터프레임을 CSV 파일로 저장 (UTF-8-BOM으로 인코딩)
                save_to_csv(round_df, output_file_path)

                print(f"  - 데이터를 데이터프레임으로 모두 정리하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    except Exception as e:
        print("Error:", e)
        traceback.print_exc()  # 오류 발생시 오류 내용 출력

    finally:
        xl.close()  # 엑셀 파일 닫기

# 결과 출력
print("\n  * Round DataFrame (round_df)")
print("    * Round DataFrame (기본 정보):")
display(round_df[['date_time', 'golf_course', 'course_f9', 'course_b9', 'companions']].tail(10))

print("\n    * Round DataFrame (Strokes & Putts):")
selected_columns = ['date_time', 'stroke_f9', 'stroke_f9_total', 'stroke_b9', 'stroke_b9_total', 'stroke_total',
                    'putt_f9', 'putt_f9_total', 'putt_b9', 'putt_b9_total', 'putt_total']
display(round_df[selected_columns].head(10))

print("\n    * Round DataFrame (Penalty & Fairway):")
selected_columns = ['date_time', 'penalty_f9', 'penalty_f9_total', 'penalty_b9', 'penalty_b9_total', 'penalty_total',
                    'fw_hit_f9', 'fw_hit_b9']
display(round_df[selected_columns].tail(10))

print("\n  * Round DataFrame Information (round_df):")
display(round_df.info())

# ==================================================

02-2. Round Data 입력 및 설정
  - '/content/drive/MyDrive/Colab Notebooks/gScore/Data/gScore_InputData.xlsx' 파일을 읽었습니다. (2024-08-08 13:06:52)
  - 'Round_Data' 시트를 읽었습니다. (2024-08-08 13:06:52)
  - Round Data를 읽었습니다. (2024-08-08 13:06:52)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Output/02_Round_Data_20240808.130652.csv' 파일에 데이터를 저장하였습니다. (2024-08-08 13:06:52)
  - 데이터를 데이터프레임으로 모두 정리하였습니다. (2024-08-08 13:06:52)

  * Round DataFrame (round_df)
    * Round DataFrame (기본 정보):


,date_time,golf_course,course_f9,course_b9,companions
82,2024-04-15 13:37:00,안성베네스트,East,South,"[김승범, 김세준, 이호]"
83,2024-04-25 06:51:00,해비치 제주,SKY,PALM,"[정철화, 박신형, 박호철]"
84,2024-05-17 07:14:00,써닝포인트,Sun,Point,"[주성갑, 강영균, 김도훈]"
85,2024-05-27 07:55:00,비에이비스타,Bella,Monti,"[박호철, 강영균, 임준범]"
86,2024-06-01 05:14:00,파주,West,East,"[조관희, 김경훈, 윤성원]"
87,2024-06-12 07:30:00,뉴스프링빌,록키,올림프스,"[박신형, 김도훈, 강영균]"
88,2024-07-05 07:46:00,강남300,OUT,IN,"[형한우, 정철화, 박효철]"
89,2024-07-06 06:31:00,세라지오,세라,지오,"[김준영, 박재진, 김태관]"
90,2024-07-19 11:53:00,한림용인,다이아몬드,사파이어,"[김영진, 김강휘, 김세준]"
91,2024-08-07 08:37:00,한성,블루,오렌지,"[김승범, 오철, 이인호]"



    * Round DataFrame (Strokes & Putts):


,date_time,stroke_f9,stroke_f9_total,stroke_b9,stroke_b9_total,stroke_total,putt_f9,putt_f9_total,putt_b9,putt_b9_total,putt_total
0,2021-09-10 12:06:00,"[4, 5, 7, 8, 5, 7, 7, 7, 7]",57,"[5, 5, 6, 8, 5, 7, 6, 8, 4]",54,111,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
1,2021-09-11 06:59:00,"[4, 10, 7, 5, 6, 7, 6, 4, 8]",57,"[6, 7, 5, 7, 7, 5, 8, 7, 5]",57,114,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
2,2021-10-04 07:51:00,"[4, 5, 8, 6, 6, 7, 6, 7, 9]",58,"[5, 7, 4, 6, 8, 7, 5, 7, 7]",56,114,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
3,2021-10-11 10:25:00,"[4, 5, 8, 8, 9, 5, 8, 9, 6]",62,"[7, 5, 7, 6, 9, 5, 6, 8, 6]",59,121,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
4,2021-10-15 09:00:00,"[5, 7, 6, 5, 8, 5, 7, 7, 6]",56,"[7, 6, 8, 5, 5, 8, 4, 7, 4]",54,110,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
5,2021-10-29 06:51:00,"[4, 5, 9, 6, 6, 6, 4, 8, 7]",55,"[7, 8, 6, 5, 8, 6, 6, 4, 4]",54,109,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
6,2021-10-30 08:41:00,"[7, 8, 4, 8, 6, 6, 7, 6, 7]",59,"[7, 6, 5, 7, 7, 6, 7, 5, 6]",56,115,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
7,2021-12-08 07:54:00,"[4, 8, 7, 4, 8, 6, 7, 5, 8]",57,"[9, 6, 7, 5, 6, 8, 8, 5, 6]",60,117,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
8,2022-01-15 08:09:00,"[5, 8, 6, 7, 4, 6, 8, 8, 8]",60,"[7, 7, 4, 8, 8, 8, 6, 5, 10]",63,123,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
9,2022-02-26 12:11:00,"[4, 7, 6, 7, 8, 6, 5, 6, 8]",57,"[6, 7, 5, 5, 5, 6, 7, 4, 4]",49,106,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18



    * Round DataFrame (Penalty & Fairway):


,date_time,penalty_f9,penalty_f9_total,penalty_b9,penalty_b9_total,penalty_total,fw_hit_f9,fw_hit_b9
82,2024-04-15 13:37:00,"[0, 0, 0, 1, 1, 1, 0, 0, 0]",3,"[1, 0, 1, 0, 0, 1, 0, 0, 0]",3,6,"[1, 0, 0, 1, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 1, 0, 1, 0, 0]"
83,2024-04-25 06:51:00,"[0, 0, 2, 1, 0, 0, 0, 0, 1]",4,"[0, 1, 0, 2, 1, 0, 0, 0, 0]",4,8,"[0, 0, 0, 0, 0, 0, 1, 0, 0]","[1, 0, 1, 0, 1, 0, 1, 1, 0]"
84,2024-05-17 07:14:00,"[0, 0, 3, 0, 0, 1, 1, 1, 0]",6,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,6,"[0, 0, 0, 1, 0, 0, 0, 1, 1]","[0, 0, 0, 0, 1, 0, 1, 0, 1]"
85,2024-05-27 07:55:00,"[0, 0, 2, 0, 3, 0, 3, 0, 0]",8,"[0, 1, 0, 1, 0, 0, 0, 2, 0]",4,12,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1, 1, 1, 1, 1, 1, 0, 1]"
86,2024-06-01 05:14:00,"[0, 0, 1, 1, 0, 1, 0, 0, 0]",3,"[0, 0, 0, 0, 1, 1, 1, 0, 0]",3,6,"[0, 1, 0, 0, 1, 0, 0, 0, 1]","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
87,2024-06-12 07:30:00,"[0, 1, 1, 0, 0, 0, 1, 1, 1]",5,"[2, 0, 1, 1, 0, 0, 2, 2, 0]",8,13,"[0, 0, 0, 1, 1, 1, 0, 0, 0]","[0, 1, 0, 0, 0, 1, 0, 0, 1]"
88,2024-07-05 07:46:00,"[0, 1, 1, 0, 1, 0, 0, 0, 0]",3,"[0, 0, 1, 0, 0, 2, 0, 0, 0]",3,6,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
89,2024-07-06 06:31:00,"[0, 1, 1, 1, 0, 0, 2, 1, 0]",6,"[1, 1, 0, 1, 0, 1, 0, 0, 0]",4,10,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 0, 1, 0, 1, 1, 1]"
90,2024-07-19 11:53:00,"[0, 1, 2, 0, 2, 0, 1, 0, 0]",6,"[0, 0, 0, 0, 0, 1, 1, 1, 0]",3,9,"[1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0]"
91,2024-08-07 08:37:00,"[0, 0, 2, 0, 2, 2, 0, 0, 0]",6,"[0, 0, 0, 0, 0, 0, 0, 2, 0]",2,8,"[0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 1, 0, 1, 0, 1, 0, 1, 0]"



  * Round DataFrame Information (round_df):
<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 0 to 91
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_time         92 non-null     datetime64[ns]
 1   golf_course       92 non-null     object        
 2   course_f9         92 non-null     object        
 3   course_b9         92 non-null     object        
 4   companions        92 non-null     object        
 5   stroke_f9         92 non-null     object        
 6   stroke_f9_total   92 non-null     int64         
 7   putt_f9           92 non-null     object        
 8   putt_f9_total     92 non-null     int64         
 9   penalty_f9        92 non-null     object        
 10  penalty_f9_total  92 non-null     int64         
 11  fw_hit_f9         92 non-null     object        
 12  stroke_b9         92 non-null     object        
 13  stroke_b9_total   92 non-null     int64   

None

## 02-3. 데이터 병합 및 저장

In [ ]:
# 02-3. 데이터 병합하여 gScore Data 생성 (240621)
print("02-3. 데이터 병합하여 gScore Data 생성")

# 파일 경로 및 이름 설정
output_file_path = generate_file_path('output', base_path, 'Output', '03_gScore_RawData', 'csv')  # 출력 파일 경로 및 이름 설정

# round_df에 course_df를 병합하여 gScore_df 생성
gScore_df = round_df.copy()
print(f"  - round_df를 복사하여 gScore_df를 생성하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# golf_course과 course_f9를 인덱스로 gScore_df와 course_df 매칭하여 par_f9과 par_f9_total 추가
gScore_df = gScore_df.merge(course_df[['golf_course', 'course', 'par', 'par_total', 'address']],
                            how='left',
                            left_on=['golf_course', 'course_f9'],
                            right_on=['golf_course', 'course'])
gScore_df.rename(columns={'par': 'par_f9', 'par_total': 'par_f9_total'}, inplace=True)
gScore_df.drop(columns=['course'], inplace=True)

# golf_course과 course_b9를 인덱스로 gScore_df와 course_df 매칭하여 par_b9과 par_b9_total 추가
gScore_df = gScore_df.merge(course_df[['golf_course', 'course', 'par', 'par_total']],
                            how='left',
                            left_on=['golf_course', 'course_b9'],
                            right_on=['golf_course', 'course'])
gScore_df.rename(columns={'par': 'par_b9', 'par_total': 'par_b9_total'}, inplace=True)
gScore_df.drop(columns=['course'], inplace=True)

# par_f9_total과 par_b9_total을 합쳐서 par_total 추가
gScore_df['par_total'] = gScore_df['par_f9_total'] + gScore_df['par_b9_total']

# 수정된 데이터프레임을 CSV 파일로 저장 (UTF-8-BOM으로 인코딩)
save_to_csv(gScore_df, output_file_path)

print(f"  - Course Data와 Round Data를 병합하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 결과 출력
print("\n  * gScore DataFrame (gScore_df)")
print("    * Round DataFrame (기본 정보):")
display(gScore_df[['date_time', 'golf_course', 'course_f9', 'course_b9', 'companions', 'address', 'fw_hit_f9', 'fw_hit_b9']].tail(5))

print("\n    * Round DataFrame (Strokes & Putts):")
selected_columns = ['date_time', 'stroke_f9', 'stroke_f9_total', 'stroke_b9', 'stroke_b9_total', 'stroke_total',
                    'putt_f9', 'putt_f9_total', 'putt_b9', 'putt_b9_total', 'putt_total']
display(gScore_df[selected_columns].head(5))

print("\n    * Round DataFrame (Penalty & Pars):")
selected_columns = ['date_time', 'penalty_f9', 'penalty_f9_total', 'penalty_b9', 'penalty_b9_total', 'penalty_total',
                    'par_f9', 'par_f9_total', 'par_b9', 'par_b9_total', 'par_total']
display(gScore_df[selected_columns].tail(5))

print("\n * gScore DataFrame Information (gScore_df):")
display(gScore_df.info())

# ==================================================

02-3. 데이터 병합하여 gScore Data 생성
  - round_df를 복사하여 gScore_df를 생성하였습니다. (2024-08-08 13:06:53)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Output/03_gScore_RawData_20240808.130653.csv' 파일에 데이터를 저장하였습니다. (2024-08-08 13:06:53)
  - Course Data와 Round Data를 병합하였습니다. (2024-08-08 13:06:53)

  * gScore DataFrame (gScore_df)
    * Round DataFrame (기본 정보):


,date_time,golf_course,course_f9,course_b9,companions,address,fw_hit_f9,fw_hit_b9
87,2024-06-12 07:30:00,뉴스프링빌,록키,올림프스,"[박신형, 김도훈, 강영균]",경기도 이천시 모가면 사실로527번길 158,"[0, 0, 0, 1, 1, 1, 0, 0, 0]","[0, 1, 0, 0, 0, 1, 0, 0, 1]"
88,2024-07-05 07:46:00,강남300,OUT,IN,"[형한우, 정철화, 박효철]",경기도 광주시 새말길 353,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0, 1, 0, 1, 0, 0, 0, 0, 0]"
89,2024-07-06 06:31:00,세라지오,세라,지오,"[김준영, 박재진, 김태관]",경기도 여주시 여양로 530,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 1, 0, 1, 0, 1, 1, 1]"
90,2024-07-19 11:53:00,한림용인,다이아몬드,사파이어,"[김영진, 김강휘, 김세준]",경기도 용인시 처인구 남사읍 경기동로 628,"[1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0]"
91,2024-08-07 08:37:00,한성,블루,오렌지,"[김승범, 오철, 이인호]",경기도 용인시 기흥구 구교동로 151,"[0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 1, 0, 1, 0, 1, 0, 1, 0]"



    * Round DataFrame (Strokes & Putts):


,date_time,stroke_f9,stroke_f9_total,stroke_b9,stroke_b9_total,stroke_total,putt_f9,putt_f9_total,putt_b9,putt_b9_total,putt_total
0,2021-09-10 12:06:00,"[4, 5, 7, 8, 5, 7, 7, 7, 7]",57,"[5, 5, 6, 8, 5, 7, 6, 8, 4]",54,111,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
1,2021-09-11 06:59:00,"[4, 10, 7, 5, 6, 7, 6, 4, 8]",57,"[6, 7, 5, 7, 7, 5, 8, 7, 5]",57,114,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
2,2021-10-04 07:51:00,"[4, 5, 8, 6, 6, 7, 6, 7, 9]",58,"[5, 7, 4, 6, 8, 7, 5, 7, 7]",56,114,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
3,2021-10-11 10:25:00,"[4, 5, 8, 8, 9, 5, 8, 9, 6]",62,"[7, 5, 7, 6, 9, 5, 6, 8, 6]",59,121,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18
4,2021-10-15 09:00:00,"[5, 7, 6, 5, 8, 5, 7, 7, 6]",56,"[7, 6, 8, 5, 5, 8, 4, 7, 4]",54,110,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1]",-9,-18



    * Round DataFrame (Penalty & Pars):


,date_time,penalty_f9,penalty_f9_total,penalty_b9,penalty_b9_total,penalty_total,par_f9,par_f9_total,par_b9,par_b9_total,par_total
87,2024-06-12 07:30:00,"[0, 1, 1, 0, 0, 0, 1, 1, 1]",5,"[2, 0, 1, 1, 0, 0, 2, 2, 0]",8,13,"[4, 3, 5, 4, 4, 3, 5, 4, 4]",36,"[4, 3, 4, 5, 4, 3, 4, 5, 4]",36,72
88,2024-07-05 07:46:00,"[0, 1, 1, 0, 1, 0, 0, 0, 0]",3,"[0, 0, 1, 0, 0, 2, 0, 0, 0]",3,6,"[5, 4, 3, 4, 3, 4, 4, 4, 5]",36,"[5, 4, 3, 4, 4, 4, 3, 4, 5]",36,72
89,2024-07-06 06:31:00,"[0, 1, 1, 1, 0, 0, 2, 1, 0]",6,"[1, 1, 0, 1, 0, 1, 0, 0, 0]",4,10,"[4, 5, 4, 4, 4, 3, 5, 3, 4]",36,"[4, 5, 4, 3, 4, 4, 3, 5, 4]",36,72
90,2024-07-19 11:53:00,"[0, 1, 2, 0, 2, 0, 1, 0, 0]",6,"[0, 0, 0, 0, 0, 1, 1, 1, 0]",3,9,"[4, 5, 4, 3, 5, 4, 4, 3, 4]",36,"[4, 4, 3, 5, 4, 3, 4, 5, 4]",36,72
91,2024-08-07 08:37:00,"[0, 0, 2, 0, 2, 2, 0, 0, 0]",6,"[0, 0, 0, 0, 0, 0, 0, 2, 0]",2,8,"[4, 5, 4, 3, 4, 4, 3, 5, 4]",36,"[4, 4, 5, 4, 3, 4, 3, 5, 4]",36,72



 * gScore DataFrame Information (gScore_df):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_time         92 non-null     datetime64[ns]
 1   golf_course       92 non-null     object        
 2   course_f9         92 non-null     object        
 3   course_b9         92 non-null     object        
 4   companions        92 non-null     object        
 5   stroke_f9         92 non-null     object        
 6   stroke_f9_total   92 non-null     int64         
 7   putt_f9           92 non-null     object        
 8   putt_f9_total     92 non-null     int64         
 9   penalty_f9        92 non-null     object        
 10  penalty_f9_total  92 non-null     int64         
 11  fw_hit_f9         92 non-null     object        
 12  stroke_b9         92 non-null     object        
 13  stroke_b9_total   92 non-null     in

None

In [ ]:
# 02-4. gScore Data 완성 (240621)
print("02-4. gScore Data 완성")

# 파일 경로 및 이름 설정
output_file_path = generate_file_path('output', base_path, 'Output', '04_gScore_Data', 'csv')  # 출력 파일 경로 및 이름 설정
parquet_file_path = generate_file_path('output', base_path, 'Data', 'gScore_Data', 'parquet', add_timestamp=False)  # 출력 파일 경로 및 이름 설정

# 데이터 프레임 복사
temp_df = gScore_df.copy()
print(f"  - gScore_df를 복사하여 temp_df를 생성하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 새로운 열 추가 및 데이터 계산

# score_f9 및 score_b9 계산 (Total Score; 전체 스코어)
temp_df['score_f9'] = temp_df.apply(lambda row: [row['stroke_f9'][i] if row['putt_f9'][i] == -1 else row['stroke_f9'][i] + row['penalty_f9'][i] + row['putt_f9'][i] for i in range(len(row['stroke_f9']))], axis=1)
temp_df['score_b9'] = temp_df.apply(lambda row: [row['stroke_b9'][i] if row['putt_b9'][i] == -1 else row['stroke_b9'][i] + row['penalty_b9'][i] + row['putt_b9'][i] for i in range(len(row['stroke_b9']))], axis=1)

# 퍼트가 -1인 경우 stroke_f9, stroke_b9 업데이트
temp_df['stroke_f9'] = temp_df.apply(lambda row: [-1 if row['putt_f9'][i] == -1 else row['stroke_f9'][i] for i in range(len(row['stroke_f9']))], axis=1)
temp_df['stroke_b9'] = temp_df.apply(lambda row: [-1 if row['putt_b9'][i] == -1 else row['stroke_b9'][i] for i in range(len(row['stroke_b9']))], axis=1)

# stroke_f9_total, stroke_b9_total 및 stroke_total 재계산
temp_df['stroke_f9_total'] = temp_df['stroke_f9'].apply(sum)
temp_df['stroke_b9_total'] = temp_df['stroke_b9'].apply(sum)
temp_df['stroke_total'] = temp_df['stroke_f9_total'] + temp_df['stroke_b9_total']

# score_f9_total, score_b9_total 및 score_total 계산
temp_df['score_f9_total'] = temp_df.apply(lambda row: sum(row['score_f9']), axis=1)
temp_df['score_b9_total'] = temp_df.apply(lambda row: sum(row['score_b9']), axis=1)
temp_df['score_total'] = temp_df['score_f9_total'] + temp_df['score_b9_total']
print(f"  - Score를 계산하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# sap_f9 및 sap_b9 계산 (S.A.P.; Score Against Par; 전체 스코어에서 정규 타수를 뺀 점수)
temp_df['sap_f9'] = temp_df.apply(lambda row: [row['score_f9'][i] - row['par_f9'][i] for i in range(len(row['par_f9']))], axis=1)
temp_df['sap_b9'] = temp_df.apply(lambda row: [row['score_b9'][i] - row['par_b9'][i] for i in range(len(row['par_b9']))], axis=1)

# sap_f9_total, sap_b9_total 및 sap_total 계산 (S.A.P.)
temp_df['sap_f9_total'] = temp_df.apply(lambda row: sum(row['sap_f9']), axis=1)
temp_df['sap_b9_total'] = temp_df.apply(lambda row: sum(row['sap_b9']), axis=1)
temp_df['sap_total'] = temp_df['sap_f9_total'] + temp_df['sap_b9_total']
print(f"  - S.A.P.를 계산하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# gir_f9 및 gir_b9 계산 (GiR; Greens in Regulation; 그린 적중률)
temp_df['gir_f9'] = temp_df.apply(lambda row: [((row['stroke_f9'][i] + row['penalty_f9'][i] <= row['par_f9'][i] - 2) if row['putt_f9'][i] != -1 else np.nan) for i in range(len(row['stroke_f9']))], axis=1)
temp_df['gir_b9'] = temp_df.apply(lambda row: [((row['stroke_b9'][i] + row['penalty_b9'][i] <= row['par_b9'][i] - 2) if row['putt_b9'][i] != -1 else np.nan) for i in range(len(row['stroke_b9']))], axis=1)

# gir_f9_total, gir_b9_total 및 gir_total 계산 (GiR)
temp_df['gir_f9_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['gir_f9']).any() else np.nansum(row['gir_f9']) / 9, axis=1)
temp_df['gir_b9_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['gir_b9']).any() else np.nansum(row['gir_b9']) / 9, axis=1)
temp_df['gir_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['gir_f9']).any() or np.isnan(row['gir_b9']).any() else np.nansum(row['gir_f9'] + row['gir_b9']) / 18, axis=1)
print(f"  - GiR을 계산하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# scrbl_f9 및 scrbl_b9 계산 (Scrambling; 스크램블링)
temp_df['scrbl_f9'] = temp_df.apply(lambda row: [(1 if row['gir_f9'][i] == 0 and row['sap_f9'][i] <= 0 else (0 if row['gir_f9'][i] == 0 and row['sap_f9'][i] > 0 else np.nan)) for i in range(len(row['gir_f9']))], axis=1)
temp_df['scrbl_b9'] = temp_df.apply(lambda row: [(1 if row['gir_b9'][i] == 0 and row['sap_b9'][i] <= 0 else (0 if row['gir_b9'][i] == 0 and row['sap_b9'][i] > 0 else np.nan)) for i in range(len(row['gir_b9']))], axis=1)

# scrbl_f9_total, scrbl_b9_total 및 scrbl_total 계산 (Scrambling)
temp_df['scrbl_f9_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['scrbl_f9']).all() else np.nansum(row['scrbl_f9']) / (~np.isnan(row['scrbl_f9'])).sum(), axis=1)
temp_df['scrbl_b9_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['scrbl_b9']).all() else np.nansum(row['scrbl_b9']) / (~np.isnan(row['scrbl_b9'])).sum(), axis=1)
temp_df['scrbl_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['scrbl_f9']).all() and np.isnan(row['scrbl_b9']).all() else (np.nansum(row['scrbl_f9']) + np.nansum(row['scrbl_b9'])) / ((~np.isnan(row['scrbl_f9'])).sum() + (~np.isnan(row['scrbl_b9'])).sum()), axis=1)
print(f"  - Scrambling을 계산하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# fw_hit_f9 및 fw_hit_b9 계산 (FW Hit; Fairways Hit; 페어웨이 적중률)
temp_df['fw_hit_f9'] = temp_df.apply(lambda row: [row['fw_hit_f9'][i] if row['par_f9'][i] != 3 else np.nan for i in range(len(row['par_f9']))], axis=1)
temp_df['fw_hit_b9'] = temp_df.apply(lambda row: [row['fw_hit_b9'][i] if row['par_b9'][i] != 3 else np.nan for i in range(len(row['par_b9']))], axis=1)

# fw_hit_f9_total, fw_hit_b9_total 및 fw_hit_total 계산 (FW Hit)
temp_df['fw_hit_f9_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['fw_hit_f9']).all() else np.nansum(row['fw_hit_f9']) / (~np.isnan(row['fw_hit_f9'])).sum(), axis=1)
temp_df['fw_hit_b9_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['fw_hit_b9']).all() else np.nansum(row['fw_hit_b9']) / (~np.isnan(row['fw_hit_b9'])).sum(), axis=1)
temp_df['fw_hit_total'] = temp_df.apply(lambda row: np.nan if np.isnan(row['fw_hit_f9']).all() and np.isnan(row['fw_hit_b9']).all() else (np.nansum(row['fw_hit_f9']) + np.nansum(row['fw_hit_b9'])) / ((~np.isnan(row['fw_hit_f9'])).sum() + (~np.isnan(row['fw_hit_b9'])).sum()), axis=1)
print(f"  - FW Hit을 계산하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# st_f9 및 st_b9 계산 (S.T.; Scoring Terms; 스코어 용어)
temp_df['st_f9'] = temp_df.apply(lambda row: [calculate_st(row['score_f9'][i], row['sap_f9'][i], row['par_f9'][i]) for i in range(len(row['sap_f9']))], axis=1)
temp_df['st_b9'] = temp_df.apply(lambda row: [calculate_st(row['score_b9'][i], row['sap_b9'][i], row['par_b9'][i]) for i in range(len(row['sap_b9']))], axis=1)

# st_f9_total, st_b9_total 및 st_total 계산 (S.T.)
temp_df['st_f9_total'] = ""
temp_df['st_b9_total'] = ""
temp_df['st_total'] = ""
print(f"  - Scoring Terms를 정리하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 연번 추가
temp_df['no'] = range(1, len(temp_df) + 1)

# 연도별 연번 추가
temp_df['year_no'] = temp_df.groupby(temp_df['date_time'].dt.year).cumcount() + 1
temp_df['year_no'] = temp_df['date_time'].dt.year.astype(str) + '-' + temp_df['year_no'].astype(str).str.zfill(2)

# 전체 컬럼 순서 설정
new_columns = [
    'date_time', 'no', 'year_no', 'golf_course', 'course_f9', 'course_b9', 'companions', 'address',  # 기본 정보
    'score_total', 'sap_total', 'stroke_total', 'putt_total', 'penalty_total', 'par_total', 'fw_hit_total', 'gir_total', 'scrbl_total', 'st_total',  # 전체 total 변수
    'score_f9', 'score_f9_total', 'score_b9', 'score_b9_total',  # score
    'sap_f9', 'sap_f9_total', 'sap_b9', 'sap_b9_total',  # sap
    'stroke_f9', 'stroke_f9_total', 'stroke_b9', 'stroke_b9_total',  # stroke
    'putt_f9', 'putt_f9_total', 'putt_b9', 'putt_b9_total',  # putt
    'penalty_f9', 'penalty_f9_total', 'penalty_b9', 'penalty_b9_total',  # penalty
    'par_f9', 'par_f9_total', 'par_b9', 'par_b9_total',  # par
    'st_f9', 'st_f9_total', 'st_b9', 'st_b9_total',  # sc
    'fw_hit_f9', 'fw_hit_f9_total', 'fw_hit_b9', 'fw_hit_b9_total',  # fw_hit
    'gir_f9', 'gir_f9_total', 'gir_b9', 'gir_b9_total',  # gir
    'scrbl_f9', 'scrbl_f9_total', 'scrbl_b9', 'scrbl_b9_total'  # scrbl
]

# 데이터프레임의 컬럼 순서를 변경
temp_df = temp_df[new_columns]

# 수정된 데이터프레임을 CSV 파일로 저장 (UTF-8-BOM으로 인코딩)
save_to_csv(temp_df, output_file_path)

# 파켓 파일로 저장
temp_df.to_parquet(parquet_file_path, index=False)
print(f"  * '{parquet_file_path}' 파일에 데이터를 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

print(f"  - gScore Data를 모두 정리하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 결과 출력
print("\n  * gScore DataFrame (기본 정보):")
display(temp_df[['date_time', 'no', 'year_no', 'golf_course', 'course_f9', 'course_b9', 'companions', 'address']].tail(10))

print("\n  * gScore DataFrame (Total Scores):")
selected_columns = ['date_time', 'score_total', 'sap_total', 'stroke_total', 'putt_total', 'penalty_total', 'par_total',
                    'fw_hit_total', 'gir_total', 'scrbl_total']
display(temp_df[selected_columns].head(10))

print("\n  * gScore DataFrame (Score Details):")
selected_columns = ['date_time', 'score_f9', 'score_f9_total', 'score_b9', 'score_b9_total', 'score_total']
display(temp_df[selected_columns].head(10))

print("\n  * gScore DataFrame (SAP Details):")
selected_columns = ['date_time', 'sap_f9', 'sap_f9_total', 'sap_b9', 'sap_b9_total', 'sap_total']
display(temp_df[selected_columns].tail(10))

print("\n  * gScore DataFrame (Stroke Details):")
selected_columns = ['date_time', 'stroke_f9', 'stroke_f9_total', 'stroke_b9', 'stroke_b9_total', 'stroke_total']
display(temp_df[selected_columns].head(5))
display(temp_df[selected_columns].tail(5))

print("\n  * gScore DataFrame (Putt Details):")
selected_columns = ['date_time', 'putt_f9', 'putt_f9_total', 'putt_b9', 'putt_b9_total', 'putt_total']
display(temp_df[selected_columns].head(10))

print("\n  * gScore DataFrame (Penalty Details):")
selected_columns = ['date_time', 'penalty_f9', 'penalty_f9_total', 'penalty_b9', 'penalty_b9_total', 'penalty_total']
display(temp_df[selected_columns].tail(10))

print("\n  * gScore DataFrame (Par Details):")
selected_columns = ['date_time', 'par_f9', 'par_f9_total', 'par_b9', 'par_b9_total', 'par_total']
display(temp_df[selected_columns].tail(10))

print("\n  * gScore DataFrame (Scoring Terms Details):")
selected_columns = ['date_time', 'st_f9', 'st_f9_total', 'st_b9', 'st_b9_total', 'st_total']
display(temp_df[selected_columns].tail(10))

print("\n  * gScore DataFrame (FW Hit Details):")
selected_columns = ['date_time', 'fw_hit_f9', 'fw_hit_f9_total', 'fw_hit_b9', 'fw_hit_b9_total', 'fw_hit_total']
display(temp_df[selected_columns].tail(10))

print("\n  * gScore DataFrame (GiR Details):")
selected_columns = ['date_time', 'gir_f9', 'gir_f9_total', 'gir_b9', 'gir_b9_total', 'gir_total']
display(temp_df[selected_columns].tail(10))

print("\n  * gScore DataFrame (Scramble Details):")
selected_columns = ['date_time', 'scrbl_f9', 'scrbl_f9_total', 'scrbl_b9', 'scrbl_b9_total', 'scrbl_total']
display(temp_df[selected_columns].tail(10))

print("\n * gScore DataFrame Information (gScore_df):")
display(temp_df.info())

# 임시 작업을 마친 후 원본 데이터 프레임에 결과 반영하고 임시 데이터 프레임 삭제
gScore_df = temp_df.copy()
del temp_df
print(f"\n  - 최종 Data를 gScore_df에 반영하고, temp_df를 삭제하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

02-4. gScore Data 완성
  - gScore_df를 복사하여 temp_df를 생성하였습니다. (2024-08-08 13:06:53)
  - Score를 계산하였습니다. (2024-08-08 13:06:53)
  - S.A.P.를 계산하였습니다. (2024-08-08 13:06:53)
  - GiR을 계산하였습니다. (2024-08-08 13:06:54)
  - Scrambling을 계산하였습니다. (2024-08-08 13:06:54)
  - FW Hit을 계산하였습니다. (2024-08-08 13:06:54)
  - Scoring Terms를 정리하였습니다. (2024-08-08 13:06:54)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Output/04_gScore_Data_20240808.130653.csv' 파일에 데이터를 저장하였습니다. (2024-08-08 13:06:54)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Data/gScore_Data.parquet' 파일에 데이터를 저장하였습니다. (2024-08-08 13:06:54)
  - gScore Data를 모두 정리하였습니다. (2024-08-08 13:06:54)

  * gScore DataFrame (기본 정보):


,date_time,no,year_no,golf_course,course_f9,course_b9,companions,address
82,2024-04-15 13:37:00,83,2024-09,안성베네스트,East,South,"[김승범, 김세준, 이호]",경기도 안성시 금광면 삼흥로 660
83,2024-04-25 06:51:00,84,2024-10,해비치 제주,SKY,PALM,"[정철화, 박신형, 박호철]",제주특별자치도 서귀포시 남원읍 원님로399번길 319
84,2024-05-17 07:14:00,85,2024-11,써닝포인트,Sun,Point,"[주성갑, 강영균, 김도훈]",경기도 용인시 처인구 백암면 고안로51번길 205
85,2024-05-27 07:55:00,86,2024-12,비에이비스타,Bella,Monti,"[박호철, 강영균, 임준범]",경기도 이천시 모가면 어농로 272
86,2024-06-01 05:14:00,87,2024-13,파주,West,East,"[조관희, 김경훈, 윤성원]",경기도 파주시 법원읍 화합로 306
87,2024-06-12 07:30:00,88,2024-14,뉴스프링빌,록키,올림프스,"[박신형, 김도훈, 강영균]",경기도 이천시 모가면 사실로527번길 158
88,2024-07-05 07:46:00,89,2024-15,강남300,OUT,IN,"[형한우, 정철화, 박효철]",경기도 광주시 새말길 353
89,2024-07-06 06:31:00,90,2024-16,세라지오,세라,지오,"[김준영, 박재진, 김태관]",경기도 여주시 여양로 530
90,2024-07-19 11:53:00,91,2024-17,한림용인,다이아몬드,사파이어,"[김영진, 김강휘, 김세준]",경기도 용인시 처인구 남사읍 경기동로 628
91,2024-08-07 08:37:00,92,2024-18,한성,블루,오렌지,"[김승범, 오철, 이인호]",경기도 용인시 기흥구 구교동로 151



  * gScore DataFrame (Total Scores):


,date_time,score_total,sap_total,stroke_total,putt_total,penalty_total,par_total,fw_hit_total,gir_total,scrbl_total
0,2021-09-10 12:06:00,111,39,-18,-18,-18,72,NaN,NaN,NaN
1,2021-09-11 06:59:00,114,42,-18,-18,-18,72,NaN,NaN,NaN
2,2021-10-04 07:51:00,114,41,-18,-18,-18,73,NaN,NaN,NaN
3,2021-10-11 10:25:00,121,49,-18,-18,-18,72,NaN,NaN,NaN
4,2021-10-15 09:00:00,110,38,-18,-18,-18,72,NaN,NaN,NaN
5,2021-10-29 06:51:00,109,37,-18,-18,-18,72,NaN,NaN,NaN
6,2021-10-30 08:41:00,115,43,-18,-18,-18,72,NaN,NaN,NaN
7,2021-12-08 07:54:00,117,45,-18,-18,-18,72,NaN,NaN,NaN
8,2022-01-15 08:09:00,123,51,-18,-18,-18,72,NaN,NaN,NaN
9,2022-02-26 12:11:00,106,34,-18,-18,-18,72,NaN,NaN,NaN



  * gScore DataFrame (Score Details):


...


  * gScore DataFrame (Scramble Details):


,date_time,scrbl_f9,scrbl_f9_total,scrbl_b9,scrbl_b9_total,scrbl_total
82,2024-04-15 13:37:00,"[nan, 1, 0, 0, 0, 0, 0, 0, nan]",0.142857,"[0, 0, 0, 0, nan, 0, 0, nan, nan]",0.000000,0.076923
83,2024-04-25 06:51:00,"[0, 0, 0, 0, 0, 0, nan, 0, 0]",0.000000,"[0, 0, 0, 0, 0, 0, 1, 0, 0]",0.111111,0.058824
84,2024-05-17 07:14:00,"[0, 0, 0, 1, 1, 0, 0, 0, 1]",0.333333,"[0, 0, nan, nan, nan, 0, 1, 0, 1]",0.333333,0.333333
85,2024-05-27 07:55:00,"[1, 0, 0, nan, 0, 0, 0, 0, 0]",0.125000,"[0, 0, 0, 0, 0, nan, nan, 0, nan]",0.000000,0.071429
86,2024-06-01 05:14:00,"[1, 0, 0, 0, nan, 0, 0, 1, nan]",0.285714,"[0, 0, 0, 0, 0, 1, 1, 1, 0]",0.333333,0.312500
87,2024-06-12 07:30:00,"[1, 0, 0, nan, 0, nan, 0, 0, 0]",0.142857,"[0, nan, 0, 0, 1, nan, 0, 0, 0]",0.142857,0.142857
88,2024-07-05 07:46:00,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0.000000,"[nan, nan, 0, 0, 0, 0, 0, 0, nan]",0.000000,0.000000
89,2024-07-06 06:31:00,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",0.111111,"[0, 0, 0, 0, nan, 0, nan, nan, 0]",0.000000,0.066667
90,2024-07-19 11:53:00,"[nan, 0, 0, 0, 0, nan, 0, 0, 0]",0.000000,"[0, nan, 0, 0, nan, 0, 0, 0, 0]",0.000000,0.000000
91,2024-08-07 08:37:00,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",0.111111,"[0, nan, 1, 0, 0, 0, 0, 0, nan]",0.142857,0.125000



 * gScore DataFrame Information (gScore_df):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 58 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_time         92 non-null     datetime64[ns]
 1   no                92 non-null     int64         
 2   year_no           92 non-null     object        
 3   golf_course       92 non-null     object        
 4   course_f9         92 non-null     object        
 5   course_b9         92 non-null     object        
 6   companions        92 non-null     object        
 7   address           92 non-null     object        
 8   score_total       92 non-null     int64         
 9   sap_total         92 non-null     int64         
 10  stroke_total      92 non-null     int64         
 11  putt_total        92 non-null     int64         
 12  penalty_total     92 non-null     int64         
 13  par_total         92 non-null     in

None


  - 최종 Data를 gScore_df에 반영하고, temp_df를 삭제하였습니다. (2024-08-08 13:06:56)


# 03. 배열 데이터를 개별 열로 변환 및 저장

In [ ]:
# 03. 배열 데이터 개별 열로 변환 (240621)
print("03. 배열 데이터 개별 열로 변환")

# 데이터 프레임 복사
expanded_df = gScore_df.copy()
print(f"  - gScore_df를 복사하여 expanded_df를 생성하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 배열로 되어 있는 열을 개별 열로 변환
def expand_columns(df, col_name, new_cols):
    expanded_cols = pd.DataFrame(df[col_name].tolist(), index=df.index, columns=new_cols)
    return df.drop(columns=[col_name]).join(expanded_cols)

# 분리할 열 목록 및 새로운 열 이름
columns_to_expand = {
    'companions': [f'companion_{i}' for i in range(1, 4)],
    'score_f9': [f'score_f9_{i}' for i in range(1, 10)],
    'score_b9': [f'score_b9_{i}' for i in range(1, 10)],
    'sap_f9': [f'sap_f9_{i}' for i in range(1, 10)],
    'sap_b9': [f'sap_b9_{i}' for i in range(1, 10)],
    'stroke_f9': [f'stroke_f9_{i}' for i in range(1, 10)],
    'stroke_b9': [f'stroke_b9_{i}' for i in range(1, 10)],
    'putt_f9': [f'putt_f9_{i}' for i in range(1, 10)],
    'putt_b9': [f'putt_b9_{i}' for i in range(1, 10)],
    'penalty_f9': [f'penalty_f9_{i}' for i in range(1, 10)],
    'penalty_b9': [f'penalty_b9_{i}' for i in range(1, 10)],
    'par_f9': [f'par_f9_{i}' for i in range(1, 10)],
    'par_b9': [f'par_b9_{i}' for i in range(1, 10)],
    'st_f9': [f'st_f9_{i}' for i in range(1, 10)],
    'st_b9': [f'st_b9_{i}' for i in range(1, 10)],
    'fw_hit_f9': [f'fw_hit_f9_{i}' for i in range(1, 10)],
    'fw_hit_b9': [f'fw_hit_b9_{i}' for i in range(1, 10)],
    'gir_f9': [f'gir_f9_{i}' for i in range(1, 10)],
    'gir_b9': [f'gir_b9_{i}' for i in range(1, 10)],
    'scrbl_f9': [f'scrbl_f9_{i}' for i in range(1, 10)],
    'scrbl_b9': [f'scrbl_b9_{i}' for i in range(1, 10)]
}

# 각 열에 대해 개별 열로 확장
for col, new_cols in columns_to_expand.items():
    expanded_df = expand_columns(expanded_df, col, new_cols)

print(f"  - 배열 데이터를 개별 열로 변환하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 열 순서 재정렬
columns_order = [
    'date_time', 'no', 'year_no', 'golf_course', 'course_f9', 'course_b9', 'companion_1', 'companion_2', 'companion_3',
    'score_total', 'sap_total', 'stroke_total', 'putt_total', 'penalty_total', 'par_total', 'fw_hit_total', 'gir_total', 'scrbl_total', 'st_total',
    *[f'score_f9_{i}' for i in range(1, 10)], 'score_f9_total',
    *[f'score_b9_{i}' for i in range(1, 10)], 'score_b9_total',
    *[f'sap_f9_{i}' for i in range(1, 10)], 'sap_f9_total',
    *[f'sap_b9_{i}' for i in range(1, 10)], 'sap_b9_total',
    *[f'stroke_f9_{i}' for i in range(1, 10)], 'stroke_f9_total',
    *[f'stroke_b9_{i}' for i in range(1, 10)], 'stroke_b9_total',
    *[f'putt_f9_{i}' for i in range(1, 10)], 'putt_f9_total',
    *[f'putt_b9_{i}' for i in range(1, 10)], 'putt_b9_total',
    *[f'penalty_f9_{i}' for i in range(1, 10)], 'penalty_f9_total',
    *[f'penalty_b9_{i}' for i in range(1, 10)], 'penalty_b9_total',
    *[f'par_f9_{i}' for i in range(1, 10)], 'par_f9_total',
    *[f'par_b9_{i}' for i in range(1, 10)], 'par_b9_total',
    *[f'st_f9_{i}' for i in range(1, 10)], 'st_f9_total',
    *[f'st_b9_{i}' for i in range(1, 10)], 'st_b9_total',
    *[f'fw_hit_f9_{i}' for i in range(1, 10)], 'fw_hit_f9_total',
    *[f'fw_hit_b9_{i}' for i in range(1, 10)], 'fw_hit_b9_total',
    *[f'gir_f9_{i}' for i in range(1, 10)], 'gir_f9_total',
    *[f'gir_b9_{i}' for i in range(1, 10)], 'gir_b9_total',
    *[f'scrbl_f9_{i}' for i in range(1, 10)], 'scrbl_f9_total',
    *[f'scrbl_b9_{i}' for i in range(1, 10)], 'scrbl_b9_total',
    'address'
]

expanded_df = expanded_df[columns_order]

# 'date_time' 열을 문자열로 변환
expanded_df['date_time'] = expanded_df['date_time'].astype(str)

# 결과 출력 함수
def display_last_10(df, columns):
    display(df[columns].tail(10))

# 데이터프레임의 정보 출력 함수
def display_info_in_sections(df, step=50):
    for i in range(0, len(df.columns), step):
        print(df.iloc[:, i:i+step].info())

print("\n  * Expanded gScore DataFrame (기본 정보):")
display_last_10(expanded_df, ['date_time', 'no', 'year_no', 'golf_course', 'course_f9', 'course_b9', 'companion_1', 'companion_2', 'companion_3', 'address'])

print("\n  * Expanded gScore DataFrame (Total Scores):")
total_scores_cols = ['date_time', 'score_total', 'sap_total', 'stroke_total', 'putt_total', 'penalty_total', 'par_total', 'fw_hit_total', 'gir_total', 'scrbl_total']
display_last_10(expanded_df, total_scores_cols)

print("\n  * Expanded gScore DataFrame (Score Details):")
score_details_cols = [f'score_f9_{i}' for i in range(1, 10)] + [f'score_b9_{i}' for i in range(1, 10)] + ['score_f9_total', 'score_b9_total']
display_last_10(expanded_df, ['date_time'] + score_details_cols)

print("\n  * Expanded gScore DataFrame (SAP Details):")
sap_details_cols = [f'sap_f9_{i}' for i in range(1, 10)] + [f'sap_b9_{i}' for i in range(1, 10)] + ['sap_f9_total', 'sap_b9_total']
display_last_10(expanded_df, ['date_time'] + sap_details_cols)

print("\n  * Expanded gScore DataFrame (Stroke Details):")
stroke_details_cols = [f'stroke_f9_{i}' for i in range(1, 10)] + [f'stroke_b9_{i}' for i in range(1, 10)] + ['stroke_f9_total', 'stroke_b9_total']
display_last_10(expanded_df, ['date_time'] + stroke_details_cols)

print("\n  * Expanded gScore DataFrame (Putt Details):")
putt_details_cols = [f'putt_f9_{i}' for i in range(1, 10)] + [f'putt_b9_{i}' for i in range(1, 10)] + ['putt_f9_total', 'putt_b9_total']
display_last_10(expanded_df, ['date_time'] + putt_details_cols)

print("\n  * Expanded gScore DataFrame (Penalty Details):")
penalty_details_cols = [f'penalty_f9_{i}' for i in range(1, 10)] + [f'penalty_b9_{i}' for i in range(1, 10)] + ['penalty_f9_total', 'penalty_b9_total']
display_last_10(expanded_df, ['date_time'] + penalty_details_cols)

print("\n  * Expanded gScore DataFrame (Par Details):")
par_details_cols = [f'par_f9_{i}' for i in range(1, 10)] + [f'par_b9_{i}' for i in range(1, 10)] + ['par_f9_total', 'par_b9_total']
display_last_10(expanded_df, ['date_time'] + par_details_cols)

print("\n  * Expanded gScore DataFrame (Scoring Terms Details):")
scoring_terms_cols = [f'st_f9_{i}' for i in range(1, 10)] + [f'st_b9_{i}' for i in range(1, 10)] + ['st_f9_total', 'st_b9_total']
display_last_10(expanded_df, ['date_time'] + scoring_terms_cols)

print("\n  * Expanded gScore DataFrame (FW Hit Details):")
fw_hit_details_cols = [f'fw_hit_f9_{i}' for i in range(1, 10)] + [f'fw_hit_b9_{i}' for i in range(1, 10)] + ['fw_hit_f9_total', 'fw_hit_b9_total']
display_last_10(expanded_df, ['date_time'] + fw_hit_details_cols)

print("\n  * Expanded gScore DataFrame (GiR Details):")
gir_details_cols = [f'gir_f9_{i}' for i in range(1, 10)] + [f'gir_b9_{i}' for i in range(1, 10)] + ['gir_f9_total', 'gir_b9_total']
display_last_10(expanded_df, ['date_time'] + gir_details_cols)

print("\n  * Expanded gScore DataFrame (Scramble Details):")
scramble_details_cols = [f'scrbl_f9_{i}' for i in range(1, 10)] + [f'scrbl_b9_{i}' for i in range(1, 10)] + ['scrbl_f9_total', 'scrbl_b9_total']
display_last_10(expanded_df, ['date_time'] + scramble_details_cols)

print("\n * Expanded gScore DataFrame Information (expanded_df):")
display_info_in_sections(expanded_df)

# ==================================================

03. 배열 데이터 개별 열로 변환
  - gScore_df를 복사하여 expanded_df를 생성하였습니다. (2024-08-08 13:06:56)
  - 배열 데이터를 개별 열로 변환하였습니다. (2024-08-08 13:06:56)

  * Expanded gScore DataFrame (기본 정보):


,date_time,no,year_no,golf_course,course_f9,course_b9,companion_1,companion_2,companion_3,address
82,2024-04-15 13:37:00,83,2024-09,안성베네스트,East,South,김승범,김세준,이호,경기도 안성시 금광면 삼흥로 660
83,2024-04-25 06:51:00,84,2024-10,해비치 제주,SKY,PALM,정철화,박신형,박호철,제주특별자치도 서귀포시 남원읍 원님로399번길 319
84,2024-05-17 07:14:00,85,2024-11,써닝포인트,Sun,Point,주성갑,강영균,김도훈,경기도 용인시 처인구 백암면 고안로51번길 205
85,2024-05-27 07:55:00,86,2024-12,비에이비스타,Bella,Monti,박호철,강영균,임준범,경기도 이천시 모가면 어농로 272
86,2024-06-01 05:14:00,87,2024-13,파주,West,East,조관희,김경훈,윤성원,경기도 파주시 법원읍 화합로 306
87,2024-06-12 07:30:00,88,2024-14,뉴스프링빌,록키,올림프스,박신형,김도훈,강영균,경기도 이천시 모가면 사실로527번길 158
88,2024-07-05 07:46:00,89,2024-15,강남300,OUT,IN,형한우,정철화,박효철,경기도 광주시 새말길 353
89,2024-07-06 06:31:00,90,2024-16,세라지오,세라,지오,김준영,박재진,김태관,경기도 여주시 여양로 530
90,2024-07-19 11:53:00,91,2024-17,한림용인,다이아몬드,사파이어,김영진,김강휘,김세준,경기도 용인시 처인구 남사읍 경기동로 628
91,2024-08-07 08:37:00,92,2024-18,한성,블루,오렌지,김승범,오철,이인호,경기도 용인시 기흥구 구교동로 151



  * Expanded gScore DataFrame (Total Scores):


,date_time,score_total,sap_total,stroke_total,putt_total,penalty_total,par_total,fw_hit_total,gir_total,scrbl_total
82,2024-04-15 13:37:00,95,23,50,39,6,72,0.428571,0.277778,0.076923
83,2024-04-25 06:51:00,112,40,62,42,8,72,0.214286,0.055556,0.058824
84,2024-05-17 07:14:00,95,22,54,35,6,73,0.357143,0.166667,0.333333
85,2024-05-27 07:55:00,98,26,52,34,12,72,0.428571,0.222222,0.071429
86,2024-06-01 05:14:00,92,20,54,32,6,72,0.357143,0.111111,0.312500
87,2024-06-12 07:30:00,98,26,52,33,13,72,0.214286,0.222222,0.142857
88,2024-07-05 07:46:00,102,30,58,38,6,72,0.214286,0.166667,0.000000
89,2024-07-06 06:31:00,100,28,54,36,10,72,0.357143,0.166667,0.066667
90,2024-07-19 11:53:00,107,35,58,40,9,72,0.214286,0.222222,0.000000
91,2024-08-07 08:37:00,104,32,61,35,8,72,0.357143,0.111111,0.125000



  * Expanded gScore DataFrame (Score Details):


...


  * Expanded gScore DataFrame (GiR Details):


,date_time,gir_f9_1,gir_f9_2,gir_f9_3,gir_f9_4,gir_f9_5,...,gir_b9_6,gir_b9_7,gir_b9_8,gir_b9_9,gir_f9_total,gir_b9_total
82,2024-04-15 13:37:00,True,False,False,False,False,...,False,False,True,True,0.222222,0.333333
83,2024-04-25 06:51:00,False,False,False,False,False,...,False,False,False,False,0.111111,0.000000
84,2024-05-17 07:14:00,False,False,False,False,False,...,False,False,False,False,0.000000,0.333333
85,2024-05-27 07:55:00,False,False,False,True,False,...,True,True,False,True,0.111111,0.333333
86,2024-06-01 05:14:00,False,False,False,False,True,...,False,False,False,False,0.222222,0.000000
87,2024-06-12 07:30:00,False,False,False,True,False,...,True,False,False,False,0.222222,0.222222
88,2024-07-05 07:46:00,False,False,False,False,False,...,False,False,False,True,0.000000,0.333333
89,2024-07-06 06:31:00,False,False,False,False,False,...,False,True,True,False,0.000000,0.333333
90,2024-07-19 11:53:00,True,False,False,False,False,...,False,False,False,False,0.222222,0.222222
91,2024-08-07 08:37:00,False,False,False,False,False,...,False,False,False,True,0.000000,0.222222



  * Expanded gScore DataFrame (Scramble Details):


,date_time,scrbl_f9_1,scrbl_f9_2,scrbl_f9_3,scrbl_f9_4,scrbl_f9_5,...,scrbl_b9_6,scrbl_b9_7,scrbl_b9_8,scrbl_b9_9,scrbl_f9_total,scrbl_b9_total
82,2024-04-15 13:37:00,NaN,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.142857,0.000000
83,2024-04-25 06:51:00,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.000000,0.111111
84,2024-05-17 07:14:00,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.333333,0.333333
85,2024-05-27 07:55:00,1.0,0.0,0.0,NaN,0.0,...,NaN,NaN,0.0,NaN,0.125000,0.000000
86,2024-06-01 05:14:00,1.0,0.0,0.0,0.0,NaN,...,1.0,1.0,1.0,0.0,0.285714,0.333333
87,2024-06-12 07:30:00,1.0,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.142857,0.142857
88,2024-07-05 07:46:00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.000000,0.000000
89,2024-07-06 06:31:00,1.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.111111,0.000000
90,2024-07-19 11:53:00,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000
91,2024-08-07 08:37:00,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.111111,0.142857



 * Expanded gScore DataFrame Information (expanded_df):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 50 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date_time       92 non-null     object 
 1   no              92 non-null     int64  
 2   year_no         92 non-null     object 
 3   golf_course     92 non-null     object 
 4   course_f9       92 non-null     object 
 5   course_b9       92 non-null     object 
 6   companion_1     92 non-null     object 
 7   companion_2     92 non-null     object 
 8   companion_3     92 non-null     object 
 9   score_total     92 non-null     int64  
 10  sap_total       92 non-null     int64  
 11  stroke_total    92 non-null     int64  
 12  putt_total      92 non-null     int64  
 13  penalty_total   92 non-null     int64  
 14  par_total       92 non-null     int64  
 15  fw_hit_total    58 non-null     float64
 16  gir_total       58 non-nu

# 04. 최종 Data 저장

In [ ]:
# 04. 최종 Data 저장 (240808)
print("04. 최종 Data 저장")

# 파일 경로 및 이름 설정
output_file_path = generate_file_path('output', base_path, 'Output', '05_gScore_Data_Expanded', 'csv')  # 출력 파일 경로 및 이름 설정
parquet_file_path = generate_file_path('output', base_path, 'Data', 'gScore_Data_Expanded', 'parquet', add_timestamp=False)  # 출력 파일 경로 및 이름 설정

if client:
    # Google Sheets 파일 열기
    spreadsheet = client.open_by_url('https://docs.google.com/spreadsheets/d/11ziQdI7gX3yZKeF2gQMvfn6krwFCDJgYdYrAsfQlKgI')
    worksheet = spreadsheet.worksheet("data")  # 'data' 시트 선택

    # NaN 값 처리 (Google Sheets에 저장하기 전에만 처리)
    df_for_gsheets = expanded_df.fillna('')

    # Google Sheets에 데이터프레임 저장
    def df_to_gsheets(df, worksheet):
        worksheet.clear()  # 기존 데이터 지우기
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())

    # 데이터프레임을 Google Sheets에 저장
    df_to_gsheets(df_for_gsheets, worksheet)
    print(f"  * Google Sheets에 데이터를 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
else:
    print("  - Google Sheets API 클라이언트가 정의되지 않았습니다. Google Sheets 저장을 건너뜁니다.")

# 데이터프레임을 CSV 파일로 저장 (UTF-8-BOM으로 인코딩)
save_to_csv(expanded_df, output_file_path)

# Parquet 파일로 저장
expanded_df.to_parquet(parquet_file_path, index=False)
print(f"  * '{parquet_file_path}' 파일에 데이터를 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

print(f"\n  * 모든 데이터 저장을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

04. 최종 Data 저장
  * Google Sheets에 데이터를 저장하였습니다. (2024-08-08 13:07:01)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Output/05_gScore_Data_Expanded_20240808.130657.csv' 파일에 데이터를 저장하였습니다. (2024-08-08 13:07:01)
  * '/content/drive/MyDrive/Colab Notebooks/gScore/Data/gScore_Data_Expanded.parquet' 파일에 데이터를 저장하였습니다. (2024-08-08 13:07:02)

  * 모든 데이터 저장을 완료하였습니다. (2024-08-08 13:07:02)
